TEST

In [1]:

import ruptures as rpt
import numpy as np
import pandas as pd

In [2]:
import data_processing
import feature_extraction

In [3]:
df = data_processing.read_all_users('geolife_trajectories')
df = data_processing.process_classes(df)

[1/182] processing user 135
[2/182] processing user 132
[3/182] processing user 104
[4/182] processing user 103
[5/182] processing user 168
[6/182] processing user 157
[7/182] processing user 150
[8/182] processing user 159
[9/182] processing user 166
[10/182] processing user 161
[11/182] processing user 102
[12/182] processing user 105
[13/182] processing user 133
[14/182] processing user 134
[15/182] processing user 160
[16/182] processing user 158
[17/182] processing user 167
[18/182] processing user 151
[19/182] processing user 169
[20/182] processing user 156
[21/182] processing user 024
[22/182] processing user 023
[23/182] processing user 015
[24/182] processing user 012
[25/182] processing user 079
[26/182] processing user 046
[27/182] processing user 041
[28/182] processing user 048
[29/182] processing user 077
[30/182] processing user 083
[31/182] processing user 084
[32/182] processing user 070
[33/182] processing user 013
[34/182] processing user 014
[35/182] processing use

In [4]:
unlabelled_df = data_processing.get_unlabelled(df)

In [5]:
unlabelled_df = data_processing.create_segments(df)

In [6]:
first_5000_segments = unlabelled_df.iloc[:5000]

In [7]:
unlabelled_df = data_processing.calculations(unlabelled_df)

In [8]:
unlabelled_df = unlabelled_df.iloc[:50000]

In [ ]:
unlabelled_df

,lat,lon,alt,elapsed time,time,trajectory,label,user,segment,distance,speed,time_diff,acceleration,bearing,heading_change,Vrate,angular_velocity,angular_acceleration
3,39.974320,116.399588,492.0,39816.056690,2009-01-03 01:21:38,20090103012134.plt,0,135,1,0.005672,0.002836,2.0,-0.001670,77.547097,210.275581,1.177637,-105.137790,-61.981946
4,39.974365,116.399730,491.0,39816.056701,2009-01-03 01:21:39,20090103012134.plt,0,135,1,0.013094,0.013094,1.0,0.010258,67.533389,10.013708,0.783402,-10.013708,95.124082
5,39.974391,116.399782,491.0,39816.056736,2009-01-03 01:21:42,20090103012134.plt,0,135,1,0.005291,0.001764,3.0,-0.003777,56.877188,10.656201,6.424578,-3.552067,2.153880
6,39.974426,116.399735,491.0,39816.056782,2009-01-03 01:21:46,20090103012134.plt,0,135,1,0.005584,0.001396,4.0,-0.000092,314.179106,257.301918,0.263217,64.325480,16.969387
7,39.974458,116.399700,491.0,39816.056840,2009-01-03 01:21:51,20090103012134.plt,0,135,1,0.004643,0.000929,5.0,-0.000094,320.031126,5.852020,0.503513,1.170404,-12.631015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38810,30.505543,114.297375,27.0,39837.849641,2009-01-24 20:23:29,20090124065103.plt,0,135,15,0.010948,0.010948,1.0,-0.000120,185.523781,0.438226,0.010981,-0.438226,0.551046
38811,30.505448,114.297366,26.0,39837.849653,2009-01-24 20:23:30,20090124065103.plt,0,135,15,0.010599,0.010599,1.0,-0.000349,184.666335,0.857446,0.032956,-0.857446,-0.419220
38812,30.505357,114.297362,26.0,39837.849664,2009-01-24 20:23:31,20090124065103.plt,0,135,15,0.010126,0.010126,1.0,-0.000473,182.168853,2.497481,0.046678,-2.497481,-1.640035
38813,30.505267,114.297359,26.0,39837.849676,2009-01-24 20:23:32,20090124065103.plt,0,135,15,0.010012,0.010012,1.0,-0.000114,181.645049,0.523804,0.011419,-0.523804,1.973677


: 

In [ ]:
import ruptures as rpt
import pandas as pd

def detect_change_points(group, penalty=5):
    """
    Detects change points within a single segment using PELT with 'rbf'.

    Parameters:
    - group: DataFrame containing data for a single segment.
    - penalty: Controls the sensitivity of segmentation.

    Returns:
    - List of detected change points (global indices).
    """
    segment_data = group[['speed', 'acceleration']].values

    if len(segment_data) < 2:  # Skip small segments
        return []

    try:
        algo = rpt.Pelt(model="rbf").fit(segment_data)
        cps = algo.predict(pen=penalty)[:-1]  # Exclude last boundary
        return group.index[cps].tolist()  # Convert to global indices
    except Exception as e:
        print(f"Error processing segment {group['segment'].iloc[0]}: {e}")
        return []

# Apply the function to each segment using groupby()
change_points_dict = unlabelled_df.groupby('segment').apply(detect_change_points)

neuer code mit nem loop

In [ ]:
import ruptures as rpt
import pandas as pd

def detect_change_points(group, penalty=5):
    """
    Detects change points within a single segment using PELT with 'rbf'.

    Parameters:
    - group: DataFrame containing data for a single segment.
    - penalty: Controls the sensitivity of segmentation.

    Returns:
    - List of detected change points (global indices).
    """
    segment_data = group[['speed', 'acceleration']].values

    if len(segment_data) < 2:  # Skip small segments
        return []

    try:
        algo = rpt.Pelt(model="rbf").fit(segment_data)
        cps = algo.predict(pen=penalty)[:-1]  # Exclude last boundary
        return group.index[cps].tolist()  # Convert to global indices
    except Exception as e:
        print(f"Error processing segment {group['segment'].iloc[0]}: {e}")
        return []

# **Controlled Processing of Segments to Prevent Crashes**
def batch_process_change_points(df, max_segments=5000, penalty=5):
    """
    Processes change point detection in batches to prevent memory overload.

    Parameters:
    - df: DataFrame with ['segment', 'speed', 'acceleration']
    - max_segments: Maximum number of segments to process.
    - penalty: Penalty for the change point detection.

    Returns:
    - Dictionary mapping segment numbers to detected change points.
    """
    change_points_dict = {}
    
    # Get the first `max_segments` unique segment IDs
    segment_ids = df['segment'].unique()[:max_segments]

    for segment, group in df[df['segment'].isin(segment_ids)].groupby('segment'):
        change_points_dict[segment] = detect_change_points(group, penalty=penalty)

    return change_points_dict

# **Apply the optimized batch processing**
change_points_dict = batch_process_change_points(unlabelled_df, max_segments=5000, penalty=5)
print("Processed segments:", len(change_points_dict))


In [11]:
change_points_dict

segment
1    [60, 350, 582, 607, 708, 853, 873, 1099, 1114,...
2                                                 [23]
3    [172, 187, 242, 288, 318, 343, 433, 454, 469, ...
4                                       [94, 224, 454]
5                                    [683, 1038, 1209]
6                                        [43, 98, 140]
dtype: object